In [7]:
import re
import pandas as pd
import numpy as np
import sqlite3
import os

# change directory 
os.chdir(r'C:\Users\Rodzinka\Desktop\siatka_23')

In [8]:
# create function for calculating rolling averages
def rolling_mean(df, list_of_n):
    df2 = pd.DataFrame()
    for n in list_of_n:
        for column in df.columns[:15]:
            new_name = 'AVG_' + str(n) + '_' + column
            df2[new_name] = df[column].rolling(n, min_periods = 1, closed='left').mean().round(2)
    df2["ID"] = df["ID"]
    df2['Team_name'] = df['Team_name']
    df2['Year'] = df['Year']
    df2['Round'] = df['Round']
    df2['Results'] = df['Result']
    df2['Date'] = df['Date']
    return df2

In [9]:
# read database, change path and database name if needed
conn = sqlite3.connect('data/database_matches_all.db')
cursor = conn.cursor()
result = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

# find table names
table_names = sorted(list(zip(*result))[0])

# define teams' names
names = ['Aluron', 'Częstochowa', 'Asseco', 'BBTS', 'Bydgoszcz', 'Kielce', 'Szczecin', 'Czarni', 'Lubin', 
         'Indykpol', 'GKS', 'Węgiel', 'Będzin', 'Wieluń',
         'Lublin', 'Skra', 'Warsza', 'Nysa', 'Suwałki', 'ZAKSA', 'Gdańsk', 
         'www']


In [13]:
# import net differences in points
conn99 = sqlite3.connect('data/database_matches_net_difference_small_points.db')
cursor99 = conn99.cursor()
result99 = cursor99.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

# find table names - match data
table_names_hh = sorted(list(zip(*result99))[0])

# save data to one table
test_name_hh = table_names_hh[0]
data_points = pd.read_sql("SELECT * FROM '{}'".format(test_name_hh), conn99)
data_points

,Points_net,ID,Team_name,Year,Round
0,-24,0,Kielce,2010/2011,1
1,24,0,Skra,2010/2011,1
2,12,1,Warsza,2010/2011,1
3,-12,1,Węgiel,2010/2011,1
4,17,2,ZAKSA,2010/2011,1
...,...,...,...,...,...
4403,12,2375,Lublin,2023/2024,10
4404,0,2376,Nysa,2023/2024,10
4405,0,2376,Lubin,2023/2024,10
4406,8,2379,Asseco,2023/2024,10


In [14]:
# create database for new data
conn2 = sqlite3.connect('data/database_matches_averages.db')

In [6]:
# data from seasons 2021/2022 and 2020/2021 looks a little bit different than from all other seasons, 
# thus two version of the code are needed

In [17]:
# define years
years = ['2023/2024','2022/2023','2021/2022', '2020/2021']

for year in years:
    for name in names: 
        
        # list for storing table names
        names_2021 = []
        
        for table_name in table_names:
            if table_name.find(year) != -1:
                if table_name.lower().find(name.lower())!= -1:
                    names_2021.append(table_name)
                    
        if not names_2021:
            continue 
            
        else:
            # sort table names
            names_2021.sort(key = lambda x: int(x.split(' ')[-1]))

            # create final table
            tab = pd.DataFrame(columns=["Points", "Serve_number", "Serve_errors", "Serve_aces", 
                               "Serve_%_effic", "Rec_number", "Rec_errors", "Rec_%_pos",
                               "Rec_%_perf", "Atack_number", "Atack_error", "Atack_blocked",
                               "Atack_%_eff", "Block_points", "ID", "Team_name", "Round", "Year", "Result", "Date"])

            # get results for each round
            for name_tab in names_2021:
                data = pd.read_sql("SELECT * FROM '{}'".format(name_tab), conn)
                data2 = data.iloc[-1:]
                data_f = pd.concat([data2.iloc[:,6], data2.iloc[:, 9:21], data2.iloc[:, 23], data2.iloc[:, -1:]], axis = 1)
                data_f.columns = ["Points", "Serve_number", "Serve_errors", "Serve_aces", 
                                       "Serve_%_effic", "Rec_number", "Rec_errors", "Rec_%_pos",
                                       "Rec_%_perf", "Atack_number", "Atack_error", "Atack_blocked",
                                       "Atack_%_eff", "Block_points", "ID"]
                data_f['Team_name'] = name
                data_f['Round'] = name_tab.split(' ')[-1]
                data_f['Year'] = year
                data_f['Date'] = data2.iloc[:,-2]
                
                # away win = 4, away loose = 5, home win = 1, home loose = 0
                if data2.columns[-3].find('Home')!=-1:
                    data_f['Result'] = data2.iloc[:, -3]
                else:
                    if data2.iloc[:, -3].any()==1:
                        data_f['Result'] = 4
                    else:
                        data_f['Result'] = 5
                
                # append results to final table
                tab = pd.concat([tab, data_f])

            # remove % signs
            tab['Serve_%_effic'] = tab['Serve_%_effic'].str.replace('%','')
            tab['Rec_%_pos'] = tab['Rec_%_pos'].str.replace('%','')
            tab['Rec_%_perf'] = tab['Rec_%_perf'].str.replace('%','')
            try:
                tab['Atack_%_eff'] = tab['Atack_%_eff'].str.replace('%','')
            except:
                pass
        
            # convert all values to numeric
            tab = tab.apply(pd.to_numeric, errors='ignore')
            
            # merge net_points with tab table
            tab = pd.merge(tab, data_points, on=["Year", "ID", "Round", "Team_name"])
            
            # change columns order
            cols = tab.columns.tolist()
            cols = cols[:-7] + cols[-1:] + cols[-7:-1]
            tab = tab[cols]
            
            # sort tables by date in order to calculate rolling means
            tab['Date'] = pd.to_datetime(tab['Date'], format="%d.%m.%Y")
            tab = tab.sort_values(by='Date')   

            # calculate rolling means
            mean_stats = rolling_mean(tab, [1,3,5])
            
            # define key - table name
            key = year + ' ' + name 
            print(key)
            
            # Save created data to sql database
            mean_stats.to_sql(key, conn2, if_exists='replace', index=False)     

2023/2024 Aluron
2023/2024 Częstochowa
2023/2024 Asseco
2023/2024 Czarni
2023/2024 Lubin
2023/2024 Indykpol
2023/2024 GKS
2023/2024 Węgiel
2023/2024 Lublin
2023/2024 Skra
2023/2024 Warsza
2023/2024 Nysa
2023/2024 Suwałki
2023/2024 ZAKSA
2023/2024 Gdańsk
2022/2023 Aluron
2022/2023 Asseco
2022/2023 BBTS
2022/2023 Czarni
2022/2023 Lubin
2022/2023 Indykpol
2022/2023 GKS
2022/2023 Węgiel
2022/2023 Lublin
2022/2023 Skra
2022/2023 Warsza
2022/2023 Nysa
2022/2023 Suwałki
2022/2023 ZAKSA
2022/2023 Gdańsk
2021/2022 Aluron
2021/2022 Asseco
2021/2022 Czarni
2021/2022 Lubin
2021/2022 Indykpol
2021/2022 GKS
2021/2022 Węgiel
2021/2022 Lublin
2021/2022 Skra
2021/2022 Warsza
2021/2022 Nysa
2021/2022 Suwałki
2021/2022 ZAKSA
2021/2022 Gdańsk
2020/2021 Aluron
2020/2021 Asseco
2020/2021 Czarni
2020/2021 Lubin
2020/2021 Indykpol
2020/2021 GKS
2020/2021 Węgiel
2020/2021 Będzin
2020/2021 Skra
2020/2021 Warsza
2020/2021 Nysa
2020/2021 Suwałki
2020/2021 ZAKSA
2020/2021 Gdańsk


In [19]:
# define years
years = ['2019/2020', '2018/2019', '2017/2018', '2016/2017', '2015/2016', '2014/2015', '2013/2014', 
        '2012/2013', '2011/2012', '2010/2011']

for year in years:
    for name in names: 
        
        # list for storing table names
        names_2021 = []
        
        for table_name in table_names:
            if table_name.find(year) != -1:
                if table_name.lower().find(name.lower())!= -1:
                    names_2021.append(table_name)
                    
        if not names_2021:
            continue 
            
        else:

            # sort table names
            names_2021.sort(key = lambda x: int(x.split(' ')[-1]))
            
            # create final table
            tab = pd.DataFrame(columns=["Points", "Serve_number", "Serve_errors", "Serve_aces", 
                               "Serve_%_effic", "Rec_number", "Rec_errors", "Rec_%_pos",
                               "Rec_%_perf", "Atack_number", "Atack_error", "Atack_blocked",
                               "Atack_%_eff", "Block_points", "ID", "Team_name", "Year", "Round", "Date"])

            # get results for each round
            for name_tab in names_2021:
                data = pd.read_sql("SELECT * FROM '{}'".format(name_tab), conn)
                
                data2 = data.iloc[-1:]            
                data_f = pd.concat([data2.iloc[:, 6], data2.iloc[:, 7], data2.iloc[:, 9], data2.iloc[:, 8], 
                                    data2.iloc[:, 10], data2.iloc[:, 11], data2.iloc[:, 12], data2.iloc[:, 15],
                                    data2.iloc[:, 17], data2.iloc[:, 18], data2.iloc[:, 19], data2.iloc[:, 20], 
                                    data2.iloc[:, 22], data2.iloc[:, 23], data2.iloc[:, -1:]], axis = 1)

                data_f.columns = ["Points", "Serve_number", "Serve_errors", "Serve_aces", 
                                   "Serve_%_effic", "Rec_number", "Rec_errors", "Rec_%_pos",
                                   "Rec_%_perf", "Atack_number", "Atack_error", "Atack_blocked",
                                   "Atack_%_eff", "Block_points", "ID"]

                data_f['Team_name'] = name
                data_f['Year'] = year
                data_f['Round'] = name_tab.split(' ')[-1]
                data_f["Serve_%_effic"] = (data_f["Serve_aces"] - data_f["Serve_errors"])/data_f["Serve_number"]
                data_f['Date'] = data2.iloc[:,-2]
                
                # away win = 4, away loose = 5, home win = 1, home loose = 0
                if data2.columns[-3].find('Home')!=-1:
                    data_f['Result'] = data2.iloc[:, -3]
                else:
                    if data2.iloc[:, -3].any()==1:
                        data_f['Result'] = 4
                    else:
                        data_f['Result'] = 5            
                
                # append results to final table
                tab = pd.concat([tab, data_f])
                             
            # remove % signs
            try:
                tab['Serve_%_effic'] = tab['Serve_%_effic'].str.replace('%','')
            except:
                pass
            
            try:
                tab['Rec_%_pos'] = tab['Rec_%_pos'].str.replace('%','')
            except:
                pass
            
            try:
                tab['Rec_%_perf'] = tab['Rec_%_perf'].str.replace('%','')
            except:
                pass
            
            try:
                tab['Atack_%_eff'] = tab['Atack_%_eff'].str.replace('%','')
            except:
                pass            

            # convert all values to numeric
            tab = tab.apply(pd.to_numeric, errors='ignore')
            
            # merge net_points with tab table
            tab = pd.merge(tab, data_points, on=["Year", "ID", "Round", "Team_name"])
            
            # change columns order
            cols = tab.columns.tolist()
            cols = cols[:-7] + cols[-1:] + cols[-7:-1]
            tab = tab[cols]

            # sort tables by date in order to calculate rolling means
            tab['Date'] = pd.to_datetime(tab['Date'], format="%d.%m.%Y")
            tab = tab.sort_values(by='Date')   
            
            # calculate rolling means
            mean_stats = rolling_mean(tab, [1,3,5])

            # define key - table name
            key = year + ' ' + name 
            print(key)
            
            # Save created data to sql database
            mean_stats.to_sql(key, conn2, if_exists='replace', index=False)        

2019/2020 Aluron
2019/2020 Asseco
2019/2020 Bydgoszcz
2019/2020 Czarni
2019/2020 Lubin
2019/2020 Indykpol
2019/2020 GKS
2019/2020 Węgiel
2019/2020 Będzin
2019/2020 Skra
2019/2020 Warsza
2019/2020 Suwałki
2019/2020 ZAKSA
2019/2020 Gdańsk
2018/2019 Aluron
2018/2019 Asseco
2018/2019 Bydgoszcz
2018/2019 Czarni
2018/2019 Lubin
2018/2019 Indykpol
2018/2019 GKS
2018/2019 Węgiel
2018/2019 Będzin
2018/2019 Skra
2018/2019 Warsza
2018/2019 ZAKSA
2018/2019 Gdańsk
2017/2018 Aluron
2017/2018 Asseco
2017/2018 BBTS
2017/2018 Bydgoszcz
2017/2018 Kielce
2017/2018 Szczecin
2017/2018 Czarni
2017/2018 Lubin
2017/2018 Indykpol
2017/2018 GKS
2017/2018 Węgiel
2017/2018 Będzin
2017/2018 Skra
2017/2018 Warsza
2017/2018 ZAKSA
2017/2018 Gdańsk
2016/2017 Częstochowa
2016/2017 Asseco
2016/2017 BBTS
2016/2017 Bydgoszcz
2016/2017 Kielce
2016/2017 Szczecin
2016/2017 Czarni
2016/2017 Lubin
2016/2017 Indykpol
2016/2017 GKS
2016/2017 Węgiel
2016/2017 Będzin
2016/2017 Skra
2016/2017 Warsza
2016/2017 ZAKSA
2016/2017 Gdańsk

In [20]:
# close connection
cursor.close()
cursor99.close()
conn.close()
conn99.close()